# Mapping Model Inference

## Install Dependencies

In [ ]:
import os

os.chdir("/content")

repo_dir = "rg_sound_generation"
if not os.path.exists(repo_dir):
  !git clone https://github.com/TheSoundOfAIOSR/$repo_dir

os.chdir(repo_dir)
os.chdir('mapping_models')
!python -m pip -q install -e .
os.chdir('examples')

## Imports

Restart kernel after installing `mapping_models` above

In [ ]:
%matplotlib inline

import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import ddsp.training
import gin

from ddsp.training.preprocessing import F0LoudnessPreprocessor
from mapping_models.trainer import create_dataset
from IPython.display import Audio

os.chdir("/content/rg_sound_generation/mapping_models/examples")

from causal_conv_test import features_map, create_model

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Load Data and Model Checkpoints

If you don't have the checkpoints, please take a look at the following:

Training a mapping model: [Notebook](https://github.com/TheSoundOfAIOSR/rg_sound_generation/blob/main/mapping_models/examples/run_training.ipynb)

Training DDSP on Nsynth Guitar Subset: [Notebook](https://github.com/TheSoundOfAIOSR/rg_sound_generation/blob/main/members/fabio/train_ddsp_nsynth_guitar.ipynb)

In [ ]:
dataset_dir = '/content/drive/MyDrive/complete_dataset'
set_name = 'test'
batch_size = 1
mapping_model_checkpoint_path = '/content/drive/MyDrive/causal_single_stage/cp.ckpt'
gin_file_path = '/content/drive/MyDrive/ddsp_trained_30k/operative_config-30000.gin'
ddsp_model_path = '/content/drive/MyDrive/ddsp_trained_30k'
sr = 16000

In [ ]:
def map_func(features):
    inputs, outputs = features_map(features)
    return inputs, outputs, features


dataset = create_dataset(
    dataset_dir=dataset_dir,
    split=set_name,
    batch_size=batch_size,
    map_func=map_func
)

datagen = iter(dataset)

In [ ]:
mapping_model = create_model(model_type='single_stage')
_ = mapping_model.load_weights(mapping_model_checkpoint_path)
gin.parse_config_file(gin_file_path)
ddsp_model = ddsp.training.models.Autoencoder()
ddsp_model.restore(ddsp_model_path)

## Get Predictions

In [ ]:
def get_preds():
    inputs, outputs, features = next(datagen)

    f0_scaled = np.squeeze(outputs.get('f0_scaled').numpy())
    ld_scaled = np.squeeze(outputs.get('ld_scaled').numpy())

    f0_pred, ld_pred = mapping_model.predict(inputs)

    features['f0_scaled'] = tf.convert_to_tensor(f0_pred)
    features['ld_scaled'] = tf.convert_to_tensor(ld_pred)

    f0_pred = np.squeeze(f0_pred)
    ld_pred = np.squeeze(ld_pred)
    
    f0_hz, loudness_db = F0LoudnessPreprocessor.invert_scaling(f0_pred, ld_pred)
    
    features['f0_hz'] = f0_hz
    features['loudness_db'] = loudness_db

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(f0_scaled, label='f0_gt')
    plt.plot(f0_pred, label='f0_pred')
    plt.xlabel('time')
    plt.ylabel('f0_scaled')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(ld_scaled, label='ld_gt')
    plt.plot(ld_pred, label='ld_pred')
    plt.xlabel('time')
    plt.ylabel('ld_scaled')
    plt.legend()

    plt.show()

    ddsp_outputs = ddsp_model(features, training=False)
    audio_pred = ddsp_model.get_audio_from_outputs(ddsp_outputs)
    
    return features['audio'], audio_pred

In [ ]:
audio, audio_pred = get_preds()

In [ ]:
print('Original Audio')
Audio(audio, rate=sr)

In [ ]:
print('Predicted Audio')
Audio(audio_pred, rate=sr)